# **Problem Statement**  
## **27. Design a data structure for a least frequently used (LFU) cache.**

Design and implement a Least Frequently Used (LFU) Cache supporting:

- get(key) — returns the value if key exists, otherwise -1.
- put(key, value) — inserts/updates the key-value pair.

When the cache reaches capacity, it should evict the least frequently used key. If multiple keys have the same frequency, evict the least recently used key among them.

### Constraints & Example Inputs/Outputs

- Capacity is a positive integer.
- get and put should work in O(1) average time.
- Cache stores key-value pairs.

### Example 1:

```python 
# Example:
lfu = LFUCache(2)   # capacity 2
lfu.put(1, 1)
lfu.put(2, 2)
print(lfu.get(1))   # returns 1
lfu.put(3, 3)       # evicts key 2
print(lfu.get(2))   # returns -1 (not found)
print(lfu.get(3))   # returns 3
lfu.put(4, 4)       # evicts key 1
print(lfu.get(1))   # returns -1 (not found)
print(lfu.get(3))   # returns 3
print(lfu.get(4))   # returns 4


### Solution Approach

#### Understanding LFU Cache
- LFU uses frequency counts to determine eviction order.
- If frequency ties → evict the least recently used among them.
- Common approach uses:
    - A key → (value, frequency) mapping.
    - A frequency → ordered set of keys mapping.
    - A min frequency tracker to find eviction candidate.

#### Operations 
- Get(key):
    - If key exists: increase its frequency and update mappings.
    - Return value.

- Put(key, value):
    - If key exists: update value and frequency.
    - Else:
        - If capacity reached: remove LFU key.
        - Insert new key with frequency = 1.
    - Update mappings.

### Solution Code

Brute force: Store keys and count frequencies but find LFU on eviction by scanning all frequencies → O(n) eviction.

In [2]:
# Approach1: Brute Force Approach
class LFUCacheBrute:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}
        self.freq = {}

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        self.freq[key] += 1
        return self.cache[key]

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return
        if key in self.cache:
            self.cache[key] = value
            self.freq[key] += 1
            return
        if len(self.cache) >= self.capacity:
            # Evict LFU key
            min_freq = min(self.freq.values())
            keys_with_min_freq = [k for k in self.freq if self.freq[k] == min_freq]
            evict_key = keys_with_min_freq[0]
            del self.cache[evict_key]
            del self.freq[evict_key]
        self.cache[key] = value
        self.freq[key] = 1


### Alternative Solution

Optimized approach: O(1) operations using:

- HashMap for key → value, freq.
- HashMap of Doubly Linked Lists for frequency → keys.
- Min frequency tracker.

In [3]:
# Approach2: Optimized Approach (Queue BFS)
from collections import defaultdict, OrderedDict

class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.min_freq = 0
        self.key_to_val_freq = {}
        self.freq_to_keys = defaultdict(OrderedDict)

    def _update_freq(self, key):
        val, freq = self.key_to_val_freq[key]
        del self.freq_to_keys[freq][key]
        if not self.freq_to_keys[freq]:
            del self.freq_to_keys[freq]
            if self.min_freq == freq:
                self.min_freq += 1
        self.freq_to_keys[freq + 1][key] = None
        self.key_to_val_freq[key] = (val, freq + 1)

    def get(self, key: int) -> int:
        if key not in self.key_to_val_freq:
            return -1
        self._update_freq(key)
        return self.key_to_val_freq[key][0]

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return
        if key in self.key_to_val_freq:
            self.key_to_val_freq[key] = (value, self.key_to_val_freq[key][1])
            self._update_freq(key)
        else:
            if len(self.key_to_val_freq) >= self.capacity:
                evict_key, _ = self.freq_to_keys[self.min_freq].popitem(last=False)
                del self.key_to_val_freq[evict_key]
            self.key_to_val_freq[key] = (value, 1)
            self.freq_to_keys[1][key] = None
            self.min_freq = 1


### Test Cases 

In [4]:
def test_lfu_cache(cls):
    lfu = cls(2)
    lfu.put(1, 1)
    lfu.put(2, 2)
    assert lfu.get(1) == 1
    lfu.put(3, 3)      # Evicts key 2
    assert lfu.get(2) == -1
    assert lfu.get(3) == 3
    lfu.put(4, 4)      # Evicts key 1
    assert lfu.get(1) == -1
    assert lfu.get(3) == 3
    assert lfu.get(4) == 4
    print(f"All test cases passed for {cls.__name__}!")

print("Testing Brute Force LFU Cache")
test_lfu_cache(LFUCacheBrute)

print("\nTesting Optimized LFU Cache")
test_lfu_cache(LFUCache)


Testing Brute Force LFU Cache
All test cases passed for LFUCacheBrute!

Testing Optimized LFU Cache
All test cases passed for LFUCache!


## Complexity Analysis

#### Time Complexity - 

| Approach      | get      | put      |
| ------------- | -------- | -------- |
| Brute Force   | O(1)     | O(n)     |
| Optimized LFU | O(1) avg | O(1) avg |

#### Space Complexity - O(capacity)

#### Thank You!!